In [1]:
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
import requests
import datetime as dt
from concurrent.futures import ThreadPoolExecutor
import itertools

In [2]:
html = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BeautifulSoup(html.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
stock_data = pd.read_html(str(table))[0]
names = [name.replace(".", "-") for name in stock_data.Symbol]
name_query = " ".join(names)
delta_t = pd.tseries.offsets.BusinessDay(n = 21)
end_date = dt.date.today()
start_date = end_date - delta_t

In [3]:
px = yf.download(name_query, period = '1y')

[*********************100%***********************]  503 of 503 completed


In [4]:
tickers = yf.Tickers(name_query)

In [154]:
tick = yf.Ticker('AAPL')

Add in name as column for each of the below

In [161]:
tick.analyst_price_target.T.columns

Index(['targetLowPrice', 'currentPrice', 'targetMeanPrice', 'targetHighPrice',
       'numberOfAnalystOpinions'],
      dtype='object')

In [152]:
tick = tickers.tickers['AAPL']

In [145]:
def esg_retrieve(name, thread_loop, tickers):

    try:

        tick = tickers.tickers[name]
        t_sus = tick.sustainability.loc[['percentile', 'totalEsg']].T.values[0]

        thread_loop.extend([t_sus])
    
    except:
        
        thread_loop.extend([[0,0]])

In [146]:
thread_count = 25
name_count = 25

thread_loop = []

with ThreadPoolExecutor(thread_count) as executor:
    futures = [executor.submit(esg_retrieve, name, thread_loop, tickers) for name in names[:name_count]]
    

In [147]:
pd.DataFrame(thread_loop)

,0,1
0,74.68,33.61
1,40.93,24.20
2,44.01,24.98
3,55.08,27.84
4,2.21,9.71
5,21.96,18.94
6,82.67,36.40
7,5.31,12.46
8,8.15,13.97
9,6.17,12.97


In [158]:
def analyst_retrieve(name, thread_loop, tickers):

    try:

        tick = tickers.tickers[name]
        t_sus = tick.analyst_price_target.T.values[0]

        thread_loop.extend([t_sus])
    
    except:
        
        thread_loop.extend([[0,0,0,0,0]])

In [162]:
thread_count = 25
name_count = 25

thread_loop = []

analyst_cols = ['targetLowPrice', 'currentPrice', 'targetMeanPrice', 'targetHighPrice',
       'numberOfAnalystOpinions']

with ThreadPoolExecutor(thread_count) as executor:
    futures = [executor.submit(analyst_retrieve, name, thread_loop, tickers) for name in names[:name_count]]
    

df_analysts = pd.DataFrame(thread_loop, columns = analyst_cols)

In [163]:
df_analysts

,targetLowPrice,currentPrice,targetMeanPrice,targetHighPrice,numberOfAnalystOpinions
0,110.0,118.790,129.01,210.0,17.0
1,55.0,58.390,60.78,65.0,9.0
2,92.0,111.830,119.78,144.0,21.0
3,135.0,147.775,162.59,200.0,24.0
4,245.0,276.780,313.45,375.0,22.0
5,78.0,73.745,92.29,100.0,24.0
6,90.0,84.640,102.83,117.0,12.0
7,321.0,350.950,385.97,475.0,31.0
8,213.0,234.670,252.71,285.0,17.0
9,120.0,145.060,165.83,210.0,18.0
